In [3]:
import plotly.graph_objs as go
import pandas as pd
import plotly.express as px
import io
import requests
from urllib.request import urlopen
import json
import numpy as np


In [4]:
def load_dataset(tipo):
    if tipo == "Confirmed":
        file = "time_series_covid19_confirmed_global.csv"
    elif tipo == "Deaths":
        file = "time_series_covid19_deaths_global.csv"
    else:
        file = "time_series_covid19_recovered_global.csv"

    url_confirmed=f"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/{file}"
    s=requests.get(url_confirmed).content
    df=pd.read_csv(io.StringIO(s.decode('utf-8')))
    df = df.drop(["Province/State", "Lat", "Long"], axis=1)
    df = pd.DataFrame(df.set_index("Country/Region").stack()).reset_index()
    df = df.rename(columns={"Country/Region":"Country","level_1":"Date", 0:tipo})
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.groupby(["Date", "Country"])[tipo].max().reset_index()
    return df

In [5]:
df_confirmed = load_dataset("Confirmed")
df_recovered = load_dataset("Recovered")
df_deaths = load_dataset("Deaths")


new_df = pd.merge(df_confirmed, df_recovered,  how='left', left_on=["Date", "Country"], right_on = ["Date", "Country"])
df_data = pd.merge(new_df, df_deaths,  how='left', left_on=["Date", "Country"], right_on = ["Date", "Country"])

df_all = df_data.copy()

In [11]:
def generar_serie_tiempo(df_all):
    gdf = df_all.groupby(['Date', 'Country'])['Confirmed', 'Deaths', 'Recovered'].max().reset_index()
    formated_gdf = gdf.copy()
    formated_gdf = formated_gdf[formated_gdf['Country']!='Others']
    formated_gdf['Date'] = pd.to_datetime(formated_gdf['Date'])
    # formated_gdf = formated_gdf[formated_gdf["Date"] > pd.Timestamp(2020,2,1)]
    formated_gdf['Date'] = formated_gdf['Date'].dt.strftime('%m/%d/%Y')
    formated_gdf['size'] = formated_gdf['Confirmed'].pow(0.3).fillna(0)
    return formated_gdf

def generar_serie_tiempo_mapa(df_all):
    formated_gdf = generar_serie_tiempo(df_all)
    fig = px.scatter_geo(formated_gdf, locations="Country", locationmode='country names', 
                        color="Confirmed", size='size', hover_name="Country", 
                        projection="natural earth", animation_frame="Date", 
                        range_color= [0, max(formated_gdf['Confirmed'])+2])

    fig.update_layout(font=dict(
                                family="Courier New, monospace",
                                size=15,
                                color="#7f7f7f"
                            ),
                 )
        
    return fig


In [12]:
generar_serie_tiempo_mapa(df_all)

/home/davinchi/Documents/covid19/tarea11/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



https://www.kapwing.com/tools/change-video-speed

https://ezgif.com/crop-video